only Tamil dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
!pip install scikit-multilearn

     |████████████████████████████████| 89 kB 3.3 MB/s 


In [3]:
# to convert emojis to text
!pip install emot
!pip install xgboost
!pip install Textblob

     |████████████████████████████████| 61 kB 16 kB/s 


In [34]:
import numpy as np 
import pandas as pd
import re
import gc
import os
import fileinput
import string
import tensorflow as tf
import zipfile
import datetime
import sys
from tqdm  import tqdm
tqdm.pandas()
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, roc_auc_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from sklearn.metrics import classification_report
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix


from skmultilearn.ensemble import MajorityVotingClassifier
from skmultilearn.cluster import FixedLabelSpaceClusterer
from skmultilearn.problem_transform import ClassifierChain
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [ ]:
# def detect(tokens):
#     return [t for t in tokens if t in valid_forms]
    
# def replace_blank(tokens):
#     return [blank if t in valid_forms else t for t in tokens]

# def create_windows(tokens, window_size=3):
#     X = []
#     for i, word in enumerate(tokens):
#         if word == blank:
#             window = tokens[i-window_size:i] + tokens[i+1:i+window_size+1]
#             window = ' '.join(window)
#             X.append(window)    
#     return X

In [5]:
!pip install emoji

import emoji

def remove_emoji(text):
    return emoji.get_emoji_regexp().sub(u'', text)

     |████████████████████████████████| 175 kB 5.0 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=1fbb503353be170efa167016a3bcd3b87a0216b94ed256e92810308b6df7f9e9
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [6]:
# import libraries and packages

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.feature_extraction.text import  CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.ensemble import VotingClassifier 
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV 

from textblob import TextBlob

import nltk.data
import numpy as np 
import pandas as pd
import re
import time
import string

# pd.set_option("max_colwidth" ,220)
# wn = nltk.WordNetLemmatizer()
# nltk.download('wordnet')
# nltk.download('stopwords')
# stopwords = nltk.corpus.stopwords.words('english')

In [21]:
import string
import re

def clean_text(text):
    #text = ' '.join([word for word in text.split() if not word.startswith('@')])           #user name with @ is removed
    text = re.sub('@[a-zA-Z0-9]*', '', str(text))
    stop_words = open("/content/drive/MyDrive/TamilStopWords (1).txt" ,encoding='utf=8').read().split('\n')
    text = "".join([word.lower() for word in text if word not in string.punctuation]) 
    text = "".join([word for word in text if not word.isdigit()])
    tokens = re.split('\s+',text)
    text = " ".join(word for word in tokens if word not in stop_words)
    return text



data_train= pd.read_csv("/content/drive/MyDrive/Abusive Comment Detection in Tamil-ACL 2022-20220125T101444Z-001.zip (Unzipped Files)/Abusive Comment Detection in Tamil-ACL 2022/ta-en-misogyny-dev.csv",header=None,sep='\t')
data_dev= pd.read_csv("/content/drive/MyDrive/new abusive-20220218T023446Z-001/new abusive/ta-misogyny-test.csv", header=None,sep='\t')
#data_test= pd.read_csv("/content/drive/MyDrive/commaaticon_test_data (1)/test_hin.tsv", sep='\t')
data_train.columns=['label','Text']
data_dev.columns=['Text']
data_train.head()

,label,Text
0,None-of-the-above,Black saree u r so hot
1,Counter-speech,Halo first ayunga kai thatti amount keatkyrang...
2,None-of-the-above,Hello...... manithana sollunga.......... prama...
3,Xenophobia,China kaaranai nalla kalaipan daily I am enjoy...
4,None-of-the-above,Onnota ponnu ipdi Panna vituviya


In [22]:
data_train['label'].value_counts()

None-of-the-above    919
Misandry             218
Counter-speech        95
Xenophobia            70
Hope-Speech           53
Misogyny              50
Homophobia            43
Transphobic           40
Name: label, dtype: int64

In [23]:
# #file =open("/content/drive/MyDrive/Abusive Comment Detection in Tamil-ACL 2022-20220125T101444Z-001.zip (Unzipped Files)/Abusive Comment Detection in Tamil-ACL 2022/ta-misogyny-train (1).csv","r")
# read_data=file.read()
# p_w= read_data.split()
# len(p_w)

In [24]:
df_filtered = data_train.mask(lambda x: x['label'] == 'Not-Tamil')

In [25]:
df_filtered.label.value_counts()

None-of-the-above    919
Misandry             218
Counter-speech        95
Xenophobia            70
Hope-Speech           53
Misogyny              50
Homophobia            43
Transphobic           40
Name: label, dtype: int64

In [26]:
data_train.label.value_counts()

None-of-the-above    919
Misandry             218
Counter-speech        95
Xenophobia            70
Hope-Speech           53
Misogyny              50
Homophobia            43
Transphobic           40
Name: label, dtype: int64

In [27]:
data_train['clean'] = df_filtered['Text'].apply(lambda x:clean_text(x))      
data_train.head()

data_dev['clean'] = data_dev['Text'].apply(lambda x:clean_text(x))      
data_dev.head()

,Text,clean
0,அந்த தமிழன் ஒரு சாதிவெறி,தமிழன் சாதிவெறி
1,இந்த வந்தேரிக்கு தகுந்த பாடம் கற்பிப்பார்கள் த...,வந்தேரிக்கு தகுந்த பாடம் கற்பிப்பார்கள் தமிழர்கள்
2,எச்ச ராஜா நீ ஒரு மானங்கெட்ட தாயை விட நீ,எச்ச ராஜா மானங்கெட்ட தாயை
3,பணம் இருக்கும் இடத்தில் கேட்பது தவறு இல்லை பண...,பணம் கேட்பது தவறு இல்லை பணம் இல்லாத கேட்பதுதான...
4,சக தோழியாக நினைத்து ஆதரவு தாருங்கள் நண்பர்களே!...,சக தோழியாக நினைத்து ஆதரவு தாருங்கள் நண்பர்களே ...


In [28]:
#y_train=df_upsampled['label']
y_train=df_filtered['label']
#y_test=data_dev['label']


y = pd.get_dummies(df_filtered['label']).values


y_train1=pd.get_dummies(y_train).values
#y_test1=pd.get_dummies(y_test).values
diz_label = {}
for i,label in enumerate(df_filtered.label.factorize()[1]):
    diz_label[i] = label
    
diz_label

{0: 'None-of-the-above',
 1: 'Counter-speech',
 2: 'Xenophobia',
 3: 'Misandry',
 4: 'Misogyny',
 5: 'Homophobia',
 6: 'Hope-Speech',
 7: 'Transphobic'}

In [29]:
import nltk.data
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import VotingClassifier 
import pandas as pd
import re
import string
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfTransformer


count_vect = CountVectorizer()
cv_vect= count_vect.fit(data_train["clean"])
cv_vect_train= count_vect.transform(data_train["clean"])
cv_vect_test= count_vect.transform(data_dev["clean"])

tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit(cv_vect_train)
X_tfidf_train = tfidf_transformer.transform(cv_vect_train)
X_tfidf_test = tfidf_transformer.transform(cv_vect_test)


In [30]:
from imblearn.over_sampling import SMOTE
# Resampling the minority class. The strategy can be changed as required.
sm = SMOTE(sampling_strategy='minority', random_state=42)
# Fit the model to generate the data.

strategy = {0:3720, 1:3720, 2:3720, 3:3720, 4:3720, 5:3720,6:3720,7:3720}
oversample = SMOTE(sampling_strategy=strategy)
X_train, y_train = oversample.fit_resample(X_tfidf_train, y_train1)

/usr/local/lib/python3.7/dist-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (3720) in class 0 will be larger than the number of samples in the majority class (class #5 -> 919)
  f"After over-sampling, the number of samples ({n_samples})"
/usr/local/lib/python3.7/dist-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (3720) in class 1 will be larger than the number of samples in the majority class (class #5 -> 919)
  f"After over-sampling, the number of samples ({n_samples})"
/usr/local/lib/python3.7/dist-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (3720) in class 2 will be larger than the number of samples in the majority class (class #5 -> 919)
  f"After over-sampling, the number of samples ({n_samples})"
/usr/local/lib/python3.7/dist-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of sa

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp=MLPClassifier( max_iter=300,activation = 'relu',solver='adam',random_state=1)
mlp.fit(X_train,  y_train)
y_pred=mlp.predict(X_tfidf_test)

# print(classification_report(y_test1, y_pred)) 
# accuracy = accuracy_score(y_test1, y_pred)
# print("Test Accuracy:", round(accuracy*100, 4))

In [ ]:
y_prediction =np.argmax(y_pred,axis=1)

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

label = le.fit(data_train['label'])
label_t=le.fit_transform(data_train['label'])
prediction1=le.inverse_transform(y_prediction)
prediction1



In [ ]:
y_pred_df = pd.DataFrame(data=prediction1, columns=['prediction1'])
tamil = pd.DataFrame()
tamil['Text'] = data_dev['Text']
tamil['Label'] = prediction1
tamil.to_csv('MUCIC_tamil1.tsv',sep='\t',index = None)
tamil.head()

,Text,Label
0,அந்த தமிழன் ஒரு சாதிவெறி,Counter-speech
1,இந்த வந்தேரிக்கு தகுந்த பாடம் கற்பிப்பார்கள் தமிழர்கள்.,Counter-speech
2,எச்ச ராஜா நீ ஒரு மானங்கெட்ட தாயை விட நீ,Misandry
3,பணம் இருக்கும் இடத்தில் கேட்பது தவறு இல்லை பணம் இல்லாத இடத்தில் கேட்பதுதான் தவறு,Counter-speech
4,சக தோழியாக நினைத்து ஆதரவு தாருங்கள் நண்பர்களே!!!! பயனுள்ள வகையில் தகவல்களைபதிவிடுகிறேனன் பார்த்து விட்டு ஆதரவு தாருங்கள்...,Counter-speech


In [ ]:
len(data_dev),len(tamil)

(699, 699)

Using Character n_grams 

In [ ]:
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', ngram_range=(2,3))
tfidf_vect_ngram_chars.fit(data_train['clean'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(data_train['clean']) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(data_dev['clean']) 


from sklearn.neural_network import MLPClassifier
mlp=MLPClassifier(random_state=1, max_iter=300)
mlp.fit(xtrain_tfidf_ngram_chars,y_train1)
y_pred_n=mlp.predict(xvalid_tfidf_ngram_chars)

In [ ]:
y_prediction_nn =np.argmax(y_pred_n,axis=1)

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

label = le.fit(data_train['label'])
label_t=le.fit_transform(data_train['label'])
prediction2=le.inverse_transform(y_prediction_nn)
prediction2


In [ ]:
y_pred_df = pd.DataFrame(data=prediction2, columns=['prediction2'])
tamil = pd.DataFrame()
tamil['Text'] = data_dev['Text']
tamil['Label'] = prediction2
tamil.to_csv('MUCIC_tamil2.tsv',sep='\t',index = None)
tamil.head()

,Text,Label
0,அந்த தமிழன் ஒரு சாதிவெறி,None-of-the-above
1,இந்த வந்தேரிக்கு தகுந்த பாடம் கற்பிப்பார்கள் தமிழர்கள்.,Transphobic
2,எச்ச ராஜா நீ ஒரு மானங்கெட்ட தாயை விட நீ,Misandry
3,பணம் இருக்கும் இடத்தில் கேட்பது தவறு இல்லை பணம் இல்லாத இடத்தில் கேட்பதுதான் தவறு,Counter-speech
4,சக தோழியாக நினைத்து ஆதரவு தாருங்கள் நண்பர்களே!!!! பயனுள்ள வகையில் தகவல்களைபதிவிடுகிறேனன் பார்த்து விட்டு ஆதரவு தாருங்கள்...,None-of-the-above


other experiments

In [ ]:


classifier = MajorityVotingClassifier(
    clusterer = FixedLabelSpaceClusterer(clusters = [[1,2,3], [0, 2, 5], [4, 5]]),
    classifier = ClassifierChain(classifier=GaussianNB())
)
classifier.fit(X_train, y_train)

MajorityVotingClassifier(classifier=ClassifierChain(classifier=GaussianNB(),
                                                    require_dense=[True, True]),
                         clusterer=FixedLabelSpaceClusterer(clusters=[[1, 2, 3],
                                                                      [0, 2, 5],
                                                                      [4, 5]]),
                         require_dense=[False, False])

In [ ]:
y_pred_mul = classifier.predict(X_tfidf_test)
print(classification_report(y_test1, y_pred_mul)) 
accuracy = accuracy_score(y_test1, y_pred_mul)
print("Test Accuracy:", round(accuracy*100, 4))

              precision    recall  f1-score   support

           0       0.05      0.44      0.09        36
           1       0.05      0.38      0.10         8
           2       0.00      0.00      0.00        11
           3       0.25      0.42      0.32       104
           4       0.04      0.38      0.07        24
           5       0.67      0.20      0.30       346
           6       0.00      0.00      0.00         2
           7       0.00      0.00      0.00        29

   micro avg       0.16      0.25      0.19       560
   macro avg       0.13      0.23      0.11       560
weighted avg       0.46      0.25      0.26       560
 samples avg       0.19      0.25      0.21       560

Test Accuracy: 15.0


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **Model2**

In [ ]:
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', ngram_range=(2,3))
tfidf_vect_ngram_chars.fit(data_train['clean'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(data_train['clean']) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(data_dev['clean']) 

In [ ]:
#using tfidf------------>


mlp=MLPClassifier( max_iter=500,activation = 'relu',solver='adam',random_state=1)
mlp.fit(xtrain_tfidf_ngram_chars,  y_train1)
y_pred=mlp.predict(xvalid_tfidf_ngram_chars)

print(classification_report(y_test1, y_pred)) 
accuracy = accuracy_score(y_test1, y_pred)
print("Test Accuracy:", round(accuracy*100, 4))

              precision    recall  f1-score   support

           0       0.25      0.03      0.05        36
           1       0.00      0.00      0.00         8
           2       0.00      0.00      0.00        11
           3       0.69      0.46      0.55       104
           4       0.00      0.00      0.00        24
           5       0.78      0.81      0.79       346
           6       0.00      0.00      0.00         2
           7       1.00      0.03      0.07        29

   micro avg       0.76      0.59      0.66       560
   macro avg       0.34      0.17      0.18       560
weighted avg       0.68      0.59      0.60       560
 samples avg       0.59      0.59      0.59       560

Test Accuracy: 58.3929


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:

clf = RandomForestClassifier()
clf.fit(X_train,y_train)

y_pred=clf.predict(X_tfidf_test)
print(classification_report(y_test1, y_pred)) 
accuracy = accuracy_score(y_test1, y_pred)
print("Test Accuracy:", round(accuracy*100, 4))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        36
           1       0.67      0.25      0.36         8
           2       0.00      0.00      0.00        11
           3       0.57      0.40      0.47       104
           4       0.67      0.08      0.15        24
           5       0.80      0.74      0.77       346
           6       0.00      0.00      0.00         2
           7       1.00      0.07      0.13        29

   micro avg       0.75      0.54      0.63       560
   macro avg       0.46      0.19      0.24       560
weighted avg       0.69      0.54      0.58       560
 samples avg       0.54      0.54      0.54       560

Test Accuracy: 54.4643


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#using tfidf------------>
from sklearn.neural_network import MLPClassifier
mlp=MLPClassifier( max_iter=500,activation = 'relu',solver='adam',random_state=1)
mlp.fit(xtrain_tfidf_ngram_chars, xvalid_tfidf_ngram_chars )
y_pred=mlp.predict(X_tfidf_test)

print(classification_report(y_test1, y_pred)) 
accuracy = accuracy_score(y_test1, y_pred)
print("Test Accuracy:", round(accuracy*100, 4))

              precision    recall  f1-score   support

           0       0.12      0.11      0.12        36
           1       0.33      0.25      0.29         8
           2       0.11      0.36      0.16        11
           3       0.52      0.44      0.48       104
           4       0.15      0.17      0.16        24
           5       0.75      0.66      0.70       346
           6       0.50      0.50      0.50         2
           7       0.29      0.24      0.26        29

   micro avg       0.57      0.53      0.55       560
   macro avg       0.35      0.34      0.33       560
weighted avg       0.60      0.53      0.56       560
 samples avg       0.50      0.53      0.51       560

Test Accuracy: 46.4286


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:

le = LabelEncoder()

label = le.fit(data_train["label"])
label_t=le.fit_transform(data_train["label"])
prediction=le.inverse_transform(prediction_num)

In [ ]:
# with char n_gram
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(1,3), max_features=5000)
tfidf_vect_ngram_chars.fit(data_train['clean'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(data_train['clean']) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(data_dev['clean']) 

from sklearn.neural_network import MLPClassifier
mlp=MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=300,activation = 'relu',solver='adam',random_state=1)
mlp.fit(xtrain_tfidf_ngram_chars,y_train1)
y_pred=mlp.predict(xvalid_tfidf_ngram_chars)

print(classification_report(y_test1, y_pred)) 
accuracy = accuracy_score(y_test1, y_pred)
print("Test Accuracy:", round(accuracy*100, 4))

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:547: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  "The parameter 'token_pattern' will not be used"


              precision    recall  f1-score   support

           0       0.29      0.11      0.16        36
           1       0.33      0.25      0.29         8
           2       0.22      0.18      0.20        11
           3       0.55      0.52      0.53       104
           4       0.43      0.12      0.19        24
           5       0.78      0.77      0.78       346
           6       0.00      0.00      0.00         2
           7       0.56      0.17      0.26        29

   micro avg       0.69      0.60      0.65       560
   macro avg       0.39      0.27      0.30       560
weighted avg       0.66      0.60      0.62       560
 samples avg       0.59      0.60      0.60       560

Test Accuracy: 58.2143


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.neural_network import MLPClassifier
mlp=MLPClassifier()
mlp.fit(X_tfidf_train,y_train1)
y_pred=mlp.predict(X_tfidf_test)

print(classification_report(y_test1, y_pred)) 
accuracy = accuracy_score(y_test1, y_pred)
print("Test Accuracy:", round(accuracy*100, 4))

              precision    recall  f1-score   support

           0       0.33      0.19      0.25        36
           1       0.50      0.25      0.33         8
           2       0.18      0.18      0.18        11
           3       0.49      0.48      0.49       104
           4       0.12      0.08      0.10        24
           5       0.76      0.72      0.74       346
           6       0.00      0.00      0.00         2
           7       0.33      0.14      0.20        29

   micro avg       0.64      0.57      0.60       560
   macro avg       0.34      0.26      0.29       560
weighted avg       0.62      0.57      0.59       560
 samples avg       0.55      0.57      0.55       560

Test Accuracy: 53.0357


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **Next model**

In [ ]:
#from sklearn.utils import resample
#create two different dataframe of majority and minority class 
#df_majority = data_train[(data_train['label']=='Non_hope_speech')] 
#df_minority = data_train[(data_train['label']=='Hope_speech')] 
# upsample minority class
#df_minority_upsampled = resample(df_minority,replace=True, n_samples= 20778, random_state=None)  # reproducible results
# Combine majority class with upsampled minority class
#df_upsampled = pd.concat([df_minority_upsampled, df_majority])

In [ ]:
X_train=data_train['clean'].tolist()
X_test =data_dev['clean'].tolist()




In [ ]:
len(X_train)

2240

In [ ]:
vocabulary_size = 25000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
X_train1 = pad_sequences(sequences, maxlen=500)
sequences = tokenizer.texts_to_sequences(X_test)
X_test1 = pad_sequences(sequences, maxlen=500)



In [ ]:
model = Sequential()
model.add(Embedding(20000, 1000, input_length=500))
model.add(Dropout(0.2))
model.add(Conv1D(64, 2, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(8, activation='softmax'))

In [ ]:
from sklearn.utils import class_weight

In [ ]:

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:

model.fit(X_train1, y_train1,batch_size=254,
                    epochs=100,
                    verbose=1,
                    validation_split=0.1,)

In [ ]:
preds = model.predict(X_test1)

In [ ]:
prediction_num= np.argmax(preds,axis=1)
#pred=np.argmax(preds,axis=1)
#print(classification_report(np.argmax(y_test1,axis=1),np.argmax(preds,axis=1)))

In [ ]:
#y_prediction =np.argmax(y_pred,axis=1)

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

label = le.fit(data_train['label'])
label_t=le.fit_transform(data_train['label'])
prediction3=le.inverse_transform(prediction_num)


In [ ]:
y_pred_df = pd.DataFrame(data=prediction3, columns=['prediction3'])
tamil = pd.DataFrame()
tamil['Text'] = data_dev['Text']
tamil['Label'] = prediction3
tamil.to_csv('MUCIC_tamil3.tsv',sep='\t',index = None)
tamil.head()

,Text,Label
0,அந்த தமிழன் ஒரு சாதிவெறி,None-of-the-above
1,இந்த வந்தேரிக்கு தகுந்த பாடம் கற்பிப்பார்கள் தமிழர்கள்.,None-of-the-above
2,எச்ச ராஜா நீ ஒரு மானங்கெட்ட தாயை விட நீ,Misandry
3,பணம் இருக்கும் இடத்தில் கேட்பது தவறு இல்லை பணம் இல்லாத இடத்தில் கேட்பதுதான் தவறு,Counter-speech
4,சக தோழியாக நினைத்து ஆதரவு தாருங்கள் நண்பர்களே!!!! பயனுள்ள வகையில் தகவல்களைபதிவிடுகிறேனன் பார்த்து விட்டு ஆதரவு தாருங்கள்...,None-of-the-above


In [ ]:
pred=np.argmax(preds,axis=1)
print(classification_report(np.argmax(y_test1,axis=1),np.argmax(preds,axis=1)))

              precision    recall  f1-score   support

           0       0.38      0.14      0.20        36
           1       0.33      0.25      0.29         8
           2       0.07      0.09      0.08        11
           3       0.51      0.44      0.47       104
           4       0.20      0.21      0.20        24
           5       0.72      0.84      0.77       346
           6       0.00      0.00      0.00         2
           7       0.33      0.10      0.16        29

    accuracy                           0.63       560
   macro avg       0.32      0.26      0.27       560
weighted avg       0.60      0.63      0.60       560



In [31]:
# pred=np.argmax(preds,axis=1)
# print(classification_report(np.argmax(y_test1,axis=1),np.argmax(preds,axis=1)))

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

label = le.fit(data_train["label"])
label_t=le.fit_transform(data_train["label"])
prediction_1d=le.inverse_transform(prediction_num)




In [ ]:
prediction_1d

Experiments down

In [ ]:
y_pred_df = pd.DataFrame(data=prediction_1d, columns=['prediction_1d'])
tamil = pd.DataFrame()
tamil['Text'] = data_dev['Text']
tamil['label'] = prediction_1d
tamil.to_csv('MUCIC_tamil3.tsv',sep='\t',index = None)
tamil.head()

,Text,label
0,அந்த தமிழன் ஒரு சாதிவெறி,None-of-the-above
1,இந்த வந்தேரிக்கு தகுந்த பாடம் கற்பிப்பார்கள் தமிழர்கள்.,None-of-the-above
2,எச்ச ராஜா நீ ஒரு மானங்கெட்ட தாயை விட நீ,Misandry
3,பணம் இருக்கும் இடத்தில் கேட்பது தவறு இல்லை பணம் இல்லாத இடத்தில் கேட்பதுதான் தவறு,None-of-the-above
4,சக தோழியாக நினைத்து ஆதரவு தாருங்கள் நண்பர்களே!!!! பயனுள்ள வகையில் தகவல்களைபதிவிடுகிறேனன் பார்த்து விட்டு ஆதரவு தாருங்கள்...,None-of-the-above


In [ ]:
len(data_dev),len(tamil),

(699, 699)